In [1]:
import os

In [10]:
os.chdir("../")
print(os.getcwd())

e:\Wine-Quality-Prediction


In [3]:
%pwd

'e:\\Wine-Quality-Prediction'

In [14]:
import pandas as pd

In [7]:
data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv",sep=";")
data.head(1)

,"fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality"
0,"7.4,0.7,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0...."


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 1 columns):
 #   Column                                                                                                                                             Non-Null Count  Dtype 
---  ------                                                                                                                                             --------------  ----- 
 0   fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality  1599 non-null   object
dtypes: object(1)
memory usage: 12.6+ KB


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)

class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema=read_yaml(SCHEMA_FILE_PATH)
        create_directories([self.config.artifacts_root])
    
    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema=self.schema.COLUMNS
        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )
        
        return data_validation_config

In [7]:
import os
from mlProject import logger

In [12]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config=config

    def validate_all_columns(self)->bool:
        try:
            validation_status=None

            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)

            all_schema=self.config.all_schema.keys()

            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,"w") as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [15]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-10-09 22:24:08,569]:INFO:common:yaml file:config\config.yaml loaded successfully]
[2025-10-09 22:24:08,571]:INFO:common:yaml file:params.yaml loaded successfully]
[2025-10-09 22:24:08,575]:INFO:common:yaml file:schema.yaml loaded successfully]
[2025-10-09 22:24:08,571]:INFO:common:yaml file:params.yaml loaded successfully]
[2025-10-09 22:24:08,575]:INFO:common:yaml file:schema.yaml loaded successfully]
[2025-10-09 22:24:08,578]:INFO:common:created directory at:artifacts]
[2025-10-09 22:24:08,580]:INFO:common:created directory at:artifacts/data_validation]
[2025-10-09 22:24:08,578]:INFO:common:created directory at:artifacts]
[2025-10-09 22:24:08,580]:INFO:common:created directory at:artifacts/data_validation]
